In [24]:
import pandas as pd
import numpy as np
from datetime import date, timedelta

### Données hospitalières

In [5]:
df = pd.read_csv('../data/raw/donnees-hospitalieres-covid19-raw.csv', sep=';')

In [8]:
df.head(30) # Show first 30

,dep,sexe,jour,hosp,rea,rad,dc
0,01,0,2020-03-18,2,0,1,0
1,01,1,2020-03-18,1,0,1,0
2,01,2,2020-03-18,1,0,0,0
3,02,0,2020-03-18,41,10,18,11
4,02,1,2020-03-18,19,4,11,6
5,02,2,2020-03-18,22,6,7,5
6,03,0,2020-03-18,4,0,1,0
7,03,1,2020-03-18,1,0,0,0
8,03,2,2020-03-18,3,0,1,0
9,04,0,2020-03-18,3,1,2,0


In [9]:
df.describe()

,sexe,hosp,rea,rad,dc
count,46665.000000,46665.000000,46665.000000,46665.000000,46665.000000
mean,1.000000,97.539184,14.172678,384.675796,102.616501
std,0.816505,211.476959,40.426357,668.087376,189.021161
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,7.000000,0.000000,52.000000,11.000000
50%,1.000000,26.000000,2.000000,137.000000,35.000000
75%,2.000000,86.000000,10.000000,382.000000,96.000000
max,2.000000,3281.000000,855.000000,6531.000000,1797.000000


In [33]:
# create a filter for all genders
all_gender_filter = df['sexe'] == 0

In [34]:
# create a filter for yesterday's cases
yesterday = date.today() - timedelta(days = 1)
yesterday_filter = df['jour'] == yesterday.strftime('%Y-%m-%d')
yesterday_df = df.where(yesterday_filter).where(all_gender_filter) #yesterday with all genders

In [36]:
yesterday_df = yesterday_df.dropna() #remove NaN rows

In [37]:
yesterday_df.describe()

,sexe,hosp,rea,rad,dc
count,101.0,101.000000,101.000000,101.000000,101.000000
mean,0.0,47.356436,3.534653,834.623762,197.227723
std,0.0,91.979518,7.434466,1172.010963,302.271364
min,0.0,0.000000,0.000000,24.000000,1.000000
25%,0.0,3.000000,0.000000,158.000000,28.000000
50%,0.0,10.000000,1.000000,377.000000,85.000000
75%,0.0,40.000000,3.000000,840.000000,190.000000
max,0.0,529.000000,45.000000,6531.000000,1797.000000
